In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import signal

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [2]:
output_dir = Path.home() / 'Desktop' / 'disk1' / 'ecephys_data'

cache = EcephysProjectCache.from_warehouse(manifest=output_dir / 'manifest.json')

In [3]:
session_id = 715093703

In [4]:
session = cache.get_session_data(session_id)

In [5]:
session.structurewise_unit_counts

CA1      145
LP       139
grey     117
LGd       82
PO        76
VISrl     76
VISp      60
VISpm     50
VISl      42
VISam     30
APN       26
DG        26
CA3       14
PoT        1
Name: ecephys_structure_acronym, dtype: int64

In [6]:
session.probes

,description,location,sampling_rate,lfp_sampling_rate,has_lfp_data
id,,,,,
810755797,probeA,See electrode locations,29999.954846,1249.998119,True
810755799,probeB,See electrode locations,29999.906318,1249.996097,True
810755801,probeC,See electrode locations,29999.985470,1249.999395,True
810755803,probeD,See electrode locations,29999.908100,1249.996171,True
810755805,probeE,See electrode locations,29999.985679,1249.999403,True
810755807,probeF,See electrode locations,30000.028033,1250.001168,True


In [7]:
selected_probe_id = 810755799 # VISpm
# selected_probe_id = 810755801 # VISp

In [ ]:
lfp = session.get_lfp(selected_probe_id)
lfp

### Relating channel unique indices and local probe indices

In [ ]:
lfp = lfp.drop_sel(channel=850262820) # dropped because channel 850262820 is not present in cache.get_channels()

In [ ]:
lfp_channel_indices = cache.get_channels().loc[lfp.channel.values].local_index.values

In [ ]:
lfp_channel_indices

In [ ]:
temp_channels_df = cache.get_channels()
lfp_channel_dict = {temp_channels_df.loc[k].local_index.item():k  for k in lfp.channel.values}

In [ ]:
print(len(lfp_channel_dict))
print(lfp_channel_dict)

### Plotting LFP with annotated regions

In [ ]:
structure_acronyms, intervals = session.channel_structure_intervals(lfp["channel"])
interval_midpoints = [aa + (bb - aa) / 2 for aa, bb in zip(intervals[:-1], intervals[1:])]

print('FirstChannel--Structure--LastChannel')
for aa, bb, i in zip(intervals[:-1], intervals[1:], range(len(intervals))): print(f'{aa}--{structure_acronyms[i]}--{bb}')

In [ ]:
lfp_slice = lfp.sel(time=slice(129,130))

fig, ax = plt.subplots(figsize=(10,12))
pcolormesh_obj = ax.pcolormesh(lfp_slice.T)

# Plotting structure names + intervals on y-axis
ax.set_yticks(intervals)
ax.set_yticks(interval_midpoints, minor=True)
ax.set_yticklabels(structure_acronyms, minor=True)
ax.tick_params("y", which="major", labelleft=False, length=40)

# Plotting time instead of samples on x-axis
num_time_labels = 10
time_label_indices = np.around(np.linspace(1, len(lfp_slice["time"]), num_time_labels)).astype(int) - 1
time_labels = np.around(lfp_slice["time"][time_label_indices].data, decimals=1)
ax.set_xticks(time_label_indices + 0.5)
ax.set_xticklabels(time_labels)
ax.set_xlabel("time (s)", fontsize=20)

fig.colorbar(pcolormesh_obj, fraction=0.036, pad=0.04)

plt.show()

In [ ]:
lfp_slice = lfp.sel(time=slice(129,130))

channel_num = 50 # CA1
channel_id = lfp_channel_dict[list(lfp_channel_dict.keys())[channel_num]] # to get the channel ID of 50th LFP local index, not of lFP local index = 50
plt.title(f"channel number {channel_num} local probe index {list(lfp_channel_dict.keys())[channel_num]} channel ID {channel_id}")
plt.plot(lfp_slice.time, lfp_slice.sel(channel=channel_id))
plt.show()

channel_num = 70 # VISp
channel_id = lfp_channel_dict[list(lfp_channel_dict.keys())[channel_num]]
plt.title(f"channel number {channel_num} local probe index {list(lfp_channel_dict.keys())[channel_num]} channel ID {channel_id}")
plt.plot(lfp_slice.time, lfp_slice.sel(channel=channel_id))
plt.show()

In [ ]:
str_num = 1 # number of the structure from above (e.g. CA1 = 4th)

lfp_slice = lfp.sel(time=slice(129,130))

structure_traces = lfp_slice[:,intervals[str_num]:intervals[str_num+1]] # selecting all the timepoints in lfp_slice and channels within the range of i-th structure

mean_structure_trace = structure_traces.data.mean(axis=1)

plt.figure(figsize=(10,8))
plt.plot(mean_structure_trace)

# Plotting time instead of samples on x-axis
num_time_labels = 10
time_label_indices = np.around(np.linspace(1, len(lfp_slice["time"]), num_time_labels)).astype(int) - 1
time_labels = np.around(lfp_slice["time"][time_label_indices].data, decimals=1)
plt.gca().set_xticks(time_label_indices + 0.5)
plt.gca().set_xticklabels(time_labels)
plt.gca().set_xlabel("time (s)", fontsize=20)
plt.gca().set_ylabel("LFP amplitude (V)", fontsize=20)

plt.title(f'LFP trace averaged over all channels in {structure_acronyms[str_num]}', fontsize=20)

plt.show()

### Computing Current Source Density and locating layer 5 in primary visual cortex

In [ ]:
VISp_channels_vertical_positions = session.channels[(session.channels.probe_id==selected_probe_id) & (session.channels.ecephys_structure_acronym=='VISp')].probe_vertical_position
print(f'VISp veritcal extent: {VISp_channels_vertical_positions.min()}um - {VISp_channels_vertical_positions.max()}um')

In [ ]:
from scipy.ndimage.filters import gaussian_filter

csd = session.get_current_source_density(selected_probe_id)

filtered_csd = gaussian_filter(csd.data, sigma=4)

fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(20, 12))
pcolor_obj = ax[0].pcolor(csd["time"], csd["vertical_position"], filtered_csd, cmap='coolwarm_r')

ax[0].set_xlabel("time relative to stimulus onset (s)", fontsize=20)
ax[0].set_ylabel("vertical position (um)", fontsize=20)

ax[0].axhline(y=2500, color='r', linestyle='-')
ax[0].axhline(y=3420, color='r', linestyle='-')
ax[0].set_title('CSD (VISp in red)', fontsize=20)



csd_slice = csd.sel(virtual_channel_index=slice(250,350))
filtered_csd = gaussian_filter(csd_slice.data, sigma=4)
ax[1].pcolor(csd_slice["time"], csd_slice["vertical_position"], filtered_csd, cmap='coolwarm_r')
ax[1].set_xlabel("time relative to stimulus onset (s)", fontsize=20)
ax[1].set_ylabel("vertical position (um)", fontsize=20)
ax[1].set_title('CSD zoom (VISp layer 5 in blue)', fontsize=20)

ax[1].axhline(y=2500, color='r', linestyle='-')
ax[1].axhline(y=3420, color='r', linestyle='-')

ax[1].axhline(y=2950, color='b', linestyle='-')
ax[1].axhline(y=3200, color='b', linestyle='-')

fig.colorbar(pcolor_obj, fraction=0.036, pad=0.04)

plt.show()

### Looking at channels in VISp layer 5

In [ ]:
chosen_layer5 = [2950,3200]

In [ ]:
channel_df = cache.get_channels()
channel_df = channel_df[channel_df['ecephys_probe_id']==selected_probe_id]
print(len(channel_df))
channel_df.head()

These channels are within chosen layer 5 in terms of verical distance, but not all of them are associated with LFP recordings:

In [ ]:
layer5_channels_df = channel_df[(channel_df['probe_vertical_position']>=chosen_layer5[0]) & (channel_df['probe_vertical_position']<chosen_layer5[1])]
print(len(layer5_channels_df))
layer5_channels_df

Checking which channels pulled from general channel list of this probe AND in vertical range established from CSD analysis (corresponding to layer 5), are available in full LFP trace time series data:

In [ ]:
print('Channel ID -','Local Probe Index -','Channel Number')
for ch in lfp.channel.data:
    if len(layer5_channels_df[layer5_channels_df.index==ch])!=0:
        local_index = layer5_channels_df[layer5_channels_df.index==ch]['local_index'].item()
        channel_num = list(lfp_channel_dict.keys()).index(local_index)
        print(ch, local_index, channel_num)

In [ ]:
channel_ids = session.channels[(session.channels.probe_id == selected_probe_id) & \
                 (session.channels.ecephys_structure_acronym.isin(['VISpm']))].index.values

lfp_slice2 = lfp_slice.sel(channel=slice(np.min(channel_ids), np.max(channel_ids)))

fig,ax = plt.subplots(figsize=(10,8))
im = plt.imshow(lfp_slice2.T,aspect='auto',origin='lower')
# im = ax.pcolormesh(lfp_slice2.T)

fig.colorbar(im, fraction=0.036, pad=0.04)

# Plotting time instead of samples on x-axis
num_time_labels = 10
time_label_indices = np.around(np.linspace(1, len(lfp_slice2["time"]), num_time_labels)).astype(int) - 1
time_labels = np.around(lfp_slice2["time"][time_label_indices].data, decimals=1)
ax.set_xticks(time_label_indices + 0.5)
ax.set_xticklabels(time_labels)
ax.set_xlabel("time (s)", fontsize=20)

# Plotting channel IDs on y-axis
num_y_labels = len(lfp_slice2['channel'])
y_label_indices = np.around(np.linspace(1, len(lfp_slice2["channel"]), num_y_labels)).astype(int) - 1
ax.set_yticks(y_label_indices + 0.5)
ax.set_yticklabels(lfp_slice2['channel'].data)
ax.set_ylabel('Channel ID', fontsize=20)

# plotting layer 5
line1 = list(lfp_slice2['channel'].data).index(850261782) # to get the index of bordering channels
line2 = list(lfp_slice2['channel'].data).index(850261822)+1
ax.axhline(y=line1, color='b', linestyle='-')
ax.axhline(y=line2, color='b', linestyle='-')

ax.set_title('Closer look at VISp', fontsize=20)

plt.show()

### Aligning LFP to stimulus presentations

In [ ]:
stim_epochs = session.get_stimulus_epochs()
stim_epochs

In [ ]:
stim_flashes = session.get_stimulus_table(['flashes'])
# equally stim_flashes = session.stimulus_presentations[session.stimulus_presentations.stimulus_name=='flashes']
stim_flashes

In [ ]:
stim_flashes['stimulus_condition_id'].unique()

In [ ]:
presentation_times = stim_flashes.start_time.values
presentation_ids = stim_flashes.index.values

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500) # np.arange(start, stop, step)
# Trial window array (including 0.5s before and 0.5s after stimulus) is added to each timepoint linked to flash stimulus presentation
# Then all of these stim-presentation-related trial windows are concatenated into a single array
time_selection = np.concatenate([trial_window + t for t in presentation_times])

__Important to note that 1/500 increment here corresponds to downsampling to 500Hz__

In [ ]:
# This repeatedly assigns basic trial window values (-0.5s:0.5s) to every stimulus presentation id, under the column label of 'time_from_presentation_onset'
inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

In [ ]:
# Selecting LFP data at trial window constrained flash presentation times by the nearest value available
# Saving this into a newly created 'aligned_lfp' inside of xarray dataset
dataset = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
# Assigning new variabes ('presentation_id' and 'time_from_presentation_onset' from inds) to the xarray dataset
# Removing time variable
# Now, for each presentation_id, there is a corresponding time_from_presentation_onset-long array of LFP data for all the channels
dataset = dataset.assign(time=inds).unstack('time')
aligned_lfp = dataset['aligned_lfp']

In [ ]:
aligned_lfp

Channel ID - Local Probe Index - Channel Number</br>
850261782 294 73</br>
850261790 298 74</br>
850261798 302 75</br>
850261806 306 76</br>
850261814 310 77</br>
850261822 314 78</br>

In [ ]:
L5VISp_stim_aligned_lfp = aligned_lfp.sel(channel=slice(850261782, 850261822))
L5VISp_stim_aligned_lfp

In [ ]:
mean_channel_L5VISp_stim_aligned_lfp = L5VISp_stim_aligned_lfp.mean(dim='channel')
mean_channel_L5VISp_stim_aligned_lfp

### Flash stimuli

In [ ]:
stim_epochs.head()

In [ ]:
flash_index = stim_epochs[stim_epochs.stimulus_name=='flashes'].index.item()

print('No stimulus block before flashes:')
print(stim_epochs[stim_epochs.index==flash_index-1])
print('\nFlashes block:')
print(stim_epochs[stim_epochs.index==flash_index])
print('\nNo stimulus block after flashes:')
print(stim_epochs[stim_epochs.index==flash_index+1])

In [ ]:
start_time = stim_epochs[stim_epochs.index==flash_index-1].start_time.item()
end_time = stim_epochs[stim_epochs.index==flash_index+1].stop_time.item()

print(f'Looking at {start_time}s - {end_time}s time range')

In [ ]:
lfp_slice = lfp.sel(time=slice(start_time,end_time))
lfp_slice

In [ ]:
stim_flashes.head()

In [ ]:
flash_times = stim_flashes[['stimulus_condition_id','start_time','stop_time']].to_numpy()
flash_times.shape

In [ ]:
# Putting flash_times array into another format for plotting
temp_array = []
for i in range(flash_times.shape[0]):
    if flash_times[i,0]==244: val = -1
    else: val = 1
    temp_array.append([flash_times[i,1],0])
    temp_array.append([flash_times[i,1],val])
    temp_array.append([flash_times[i,2],val])
    temp_array.append([flash_times[i,2],0])
plot_format_flash_times = np.array(temp_array)

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=1,figsize=(20,16))

ax[0].set_title('full flashes block',fontsize=15)
ax[0].set_yticks([-1,0,1])
ax[0].set_yticklabels(['condition 244','grey screen','condition 245'],fontsize=15)
ax[0].plot(plot_format_flash_times[:,0],plot_format_flash_times[:,1])

ax[1].set_title('1300s-1330s zoom flashes block',fontsize=15)
ax[1].set_yticks([-1,0,1])
ax[1].set_yticklabels(['condition 244','grey screen','condition 245'],fontsize=15)
ax[1].plot(plot_format_flash_times[:,0],plot_format_flash_times[:,1])
ax[1].set_xlim([1300,1330])

plt.show()

Flash stimuli are not alternating. As described in Neuropixels Visual Coding White Paper:</br>
"The receptive field mapping stimulus was followed by a series of dark or light fullfield flashes, lasting 250 ms each and separated by a 2 second inter-trial interval."

### Spectrograms

In [ ]:
lfp_slice = lfp.sel(time=slice(start_time,end_time)).sel(channel=slice(850263324, 850263364))
lfp_slice

In [ ]:
ch = 850263348

In [ ]:
data = lfp_slice.sel(channel=ch)
# data = lfp_slice.mean(dim='channel')
data.shape

In [ ]:
print(f'LFP for channel {ch} was sampled at {layer5_channels_df[layer5_channels_df.index==ch].lfp_sampling_rate.item()} Hz')

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
stim_flashes.head()

<b>Matplotlib Specgram</b>

In [ ]:
fig,ax = plt.subplots(figsize=(20,8))

powerSpectrum, freqenciesFound, time, imageAxis = ax.specgram(data,Fs=1250,xextent=(start_time,end_time))

# ax.axvline(x=stim_flashes.iloc[0].start_time,c='r')
# ax.axvline(x=stim_flashes.iloc[-1].stop_time,c='r')

col = {244:'black',245:'white'}
for stim in stim_flashes.iterrows():
    stim = stim[1]
    ax.add_patch(Rectangle((stim.start_time,0),stim.duration,freqenciesFound.max(),facecolor=col[stim.stimulus_condition_id],alpha=0.5))

ax.set_xlabel("time (s)", fontsize=15)
ax.set_ylabel("frequency (Hz)", fontsize=15)
    
# ax.set_xlim([1300,1400])
ax.set_ylim([0,100])

ax.set_title('Matplotlib Specgram')

fig.colorbar(imageAxis, fraction=0.036, pad=0.04)

plt.show()

<b>SciPy</b>

In [ ]:
fig,ax = plt.subplots(figsize=(20,8))

frequencies, time, spectrogram = signal.spectrogram(data,fs=1250, nperseg=1024, noverlap=512, window='hann')

# ax.axvline(x=stim_flashes.iloc[0].start_time,c='r')
# ax.axvline(x=stim_flashes.iloc[-1].stop_time,c='r')

# col = {244:'black',245:'white'}
# for stim in stim_flashes.iterrows():
#     stim = stim[1]
#     ax.add_patch(Rectangle((stim.start_time,0),stim.duration,freqenciesFound.max(),facecolor=col[stim.stimulus_condition_id],alpha=1))

x_ticks_list = data.time.data
ax.set_xticks(np.linspace(0,time.shape[0],num=10))
ax.set_xticklabels(np.linspace(x_ticks_list[0],x_ticks_list[-1], num=10).astype(int))

ax.set_xlabel("time (s)", fontsize=15)
ax.set_ylabel("frequency (Hz)", fontsize=15)

# ax.set_xlim([1300,1400])
ax.set_ylim([0,100])

im = plt.imshow(10*np.log10(spectrogram),aspect='auto',origin='lower')

ax.set_title('SciPy Spectrogram')

fig.colorbar(im, fraction=0.036, pad=0.04)

plt.show()

<b>Wavelet Transform</b>

In [ ]:
# Collect flash stimulus presentations (coordinates) into array
coords = []
for stim in stim_flashes.iterrows():
    stim = stim[1]
    start_time_index = np.absolute(data.time.data - stim.start_time).argmin()
    end_time_index = np.absolute(data.time.data-(stim.start_time + stim.duration)).argmin()
    coords.append([start_time_index, 0, end_time_index, 1000, stim.stimulus_condition_id])

In [ ]:
# https://elephant.readthedocs.io/en/latest/reference/_toctree/signal_processing/elephant.signal_processing.wavelet_transform.html
from elephant.signal_processing import wavelet_transform

def plot_wt(data, freq_list, fs, plot_flashes=False):
    wt_data = wavelet_transform(data, frequency=freq_list, sampling_frequency=fs)
    
    fig,ax = plt.subplots(figsize=(20,8))
    im = ax.imshow(np.abs(wt_data)[:,:],aspect='auto',origin='lower')
    
    x_ticks_list = data.time.data

    ax.set_xticks(np.linspace(0,wt_data.shape[1],num=10))
    ax.set_xticklabels(np.linspace(x_ticks_list[0],x_ticks_list[-1], num=10).astype(int))
    ax.set_yticks(np.linspace(1,len(freq_list)-1,num=10))
    ax.set_yticklabels(np.arange(1,11))

    ax.set_xlabel("time (s)", fontsize=15)
    ax.set_ylabel("frequency (Hz)", fontsize=15)

    ax.set_title('Morlet Wavelet Transform')
    
    if plot_flashes:
        col = {244:'white',245:'green'}
        for c in coords:
            index = np.absolute(data.time.data - c[0]).argmin() # getting the index of the closest timepoint
            ax.add_patch(Rectangle((c[0],c[1]),c[2],c[3],facecolor=col[c[4]],alpha=0.5))
    # ax.add_patch(Rectangle((coords[15][0],coords[15][1]),coords[15][2],coords[15][3],facecolor=col[coords[15][4]],alpha=0.5))
    ax.add_patch(Rectangle((np.absolute(data.time.data - 1303).argmin(),0),np.absolute(data.time.data - 1306).argmin(),5,facecolor='white',alpha=1))

    fig.colorbar(im, fraction=0.036, pad=0.04)
    # plt.xlim([1300,1306])|

    plt.show()
    return wt_data

In [ ]:
freq_list = np.arange(1,100,9/50)
print(freq_list)
wt_data = plot_wt(data,freq_list,1250)

In [ ]:
freq_list = np.arange(1,10,9/50)
print(freq_list)

data1 = data.sel(time=slice(1300,1331))
wt_data1 = plot_wt(data1,freq_list,1250, plot_flashes=True)

In [ ]:
index = np.absolute(data1.time.data - 1330).argmin()
index

In [ ]:
coords[12]

In [ ]:
for i in range(0,len(coords)):
    index1 = np.absolute(data.time.data-(coords[i][0])).argmin()
    index2 = np.absolute(data.time.data-(coords[i][0]+coords[i][2])).argmin()
    print(index2-index1)

In [ ]:
time_labels = data1.time.data
time_labels.index(coords[15][0])

In [ ]:
coords[15]

In [ ]:
np.absolute(data1.time.data - coords[15][0]).argmin()

In [ ]:
data1.time.data[np.absolute(data1.time.data - coords[15][0]).argmin()]

### Power Spectral Density

In [ ]:
from elephant.spectral import welch_psd

In [ ]:
one_channel_signal = lfp_slice.sel(channel=850263348).data

In [ ]:
f, Pxx_den = signal.periodogram(one_channel_signal, fs=1250)
plt.semilogy(f, Pxx_den)
plt.xlabel('frequency (Hz)')
plt.ylabel('power (artbitrary units)')
# plt.xlim([0,30])

In [ ]:
f, psd = welch_psd(one_channel_signal, fs=1250)
# plt.semilogy(f, psd)
# plt.xlabel('frequency (Hz)')
# plt.ylabel('power (artbitrary units)')
# plt.xlim([0,30])

plt.plot(f,psd)
# plt.xlim([100,1000])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')